## CLIP (ViT-B/32) + Projection MLP + mT5-Small Decoder Pipeline

Bu bölüm: 
- CLIP ViT-L/14 image encoder (tamamen freeze)
- Görsel embedding -> K adet prefix token üreten MLP (öğrenilir)
- mT5-small (sadece decoder veya istersen tamamı) caption/çeviri üretimi
- Projection MLP ve mT5 decoder parametreleri eğitilecek.

Strateji (prefix approach):
1. Image -> CLIP encode_image -> (B,512)
2. MLP: 512 -> (K * d_model) reshape -> (B,K,512) -> LayerNorm
3. mT5 encoder'a inputs_embeds olarak bu prefix (opsiyonel ek tekst prompt tokenleri ile concat)
4. Decoder hedef yazıyı üretir (teacher forcing, cross-entropy)

Seçilebilir dondurma opsiyonları:
- freeze_clip = True (zorunlu senaryon)
- freeze_t5_encoder = True bırakıp sadece decoder + projection eğitilebilir

Aşağıdaki kod Flickr8k JSON (tasviret8k_captions.json) içinden (örnek) tek caption seçip dataset oluşturma iskeleti içerir.


In [ ]:
# # Rerun at each session start

# # Download CLIP
# !pip install -q git+https://github.com/openai/CLIP.git
# !pip install -q transformers sentencepiece accelerate
# !pip install -r kaggle/input/metric-requirements/metric_requirements.txt

# # Download mT5-small ViT-B/32
# import clip, torch
# from transformers import MT5ForConditionalGeneration, MT5Tokenizer

# device = "cuda" if torch.cuda.is_available() else "cpu"
# model_name = "google/mt5-small"

# tokenizer = MT5Tokenizer.from_pretrained(model_name)
# mt5 = MT5ForConditionalGeneration.from_pretrained(model_name).to(device)
# clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)

In [ ]:
# Unified configuration + optional wandb init
from kaggle_secrets import UserSecretsClient
import os

PROJECT_NAME = "bites-tr-image-captioning"
RUN_NAME = "clip_mt5_prefix_run"

ENABLE_WANDB = True

WANDB_API_KEY = UserSecretsClient().get_secret("WANDB_API_KEY")

base_config = {
    "model": "google/mt5-small",
    "clip_encoder": "ViT-B/32",  # backbone
    "prefix_tokens": 32,          # stronger conditioning
    "batch_size": 16,
    "lr": 1e-4,
    "epochs": 10,                  # allow a bit longer now that we fine-tune CLIP
    "dataset_limit": None,
    # --- CLIP fine-tuning controls ---
    "freeze_clip": False,          # set False to allow full CLIP fine-tuning
    "unfreeze_clip_last_n": 0,     # if >0 unfreezes only last N vision blocks (overrides freeze_clip); 0 means ignore and use freeze_clip flag
    "clip_lr_scale": 0.05,         # scaled LR for ALL CLIP params (set lower than main to avoid destroying pretrained space)
    "use_clip_patch_tokens": True, # richer conditioning (patch tokens path)
    # --- T5 freezing ---
    "freeze_t5_encoder": False,    # unfreeze encoder so it can adapt to visual prefix distribution
    "freeze_t5_decoder": False,
    # --- Optimization ---
    "seed": 42,
    "weight_decay": 0.01,
    "grad_clip": 1.0,
    "warmup_steps": 500,           # linear warmup steps before cosine decay
    # --- Inference defaults ---
    "num_beams_infer": 4,
    "max_new_tokens_infer": 32,
    "src_max_len": 64,
    "tgt_max_len": 64,
    # --- Early stopping hyperparameters ---
    "early_stop_patience": 4,
    "early_stop_min_delta": 0.01,
    
    "use_amp": True,              # force AMP on CUDA for speed
    # Optional extras:
    "use_bf16": True,
    "enable_tf32": True,
    "finite_loss_skip": True,
    "save_every": 0
}

use_wandb = False
cfg = None
if ENABLE_WANDB:
    try:
        import wandb
        wandb.login(key=WANDB_API_KEY)
        run = wandb.init(project=PROJECT_NAME, name=RUN_NAME, config=base_config)
        cfg = wandb.config
        use_wandb = True
        print("[wandb] run initialized")
    except Exception as e:
        print("[wandb] disabled (init failed):", e)

if cfg is None:
    class _Cfg: pass
    cfg = _Cfg()
    for k, v in base_config.items():
        setattr(cfg, k, v)
    print("[INFO] Using local cfg (wandb off)")

import random, numpy as np, torch
random.seed(cfg.seed)
np.random.seed(cfg.seed)
torch.manual_seed(cfg.seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(cfg.seed)

# --- Enforce CUDA-only environment ---
assert torch.cuda.is_available(), "CUDA GPU is required but not detected. Please run in a CUDA-enabled environment."
device = torch.device('cuda')
print(f"[DEVICE] Using CUDA device: {torch.cuda.get_device_name(device)}")

# Performance toggles
if getattr(cfg, 'enable_tf32', False):
    try:
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True
        print("[DEVICE] TF32 enabled.")
    except Exception as _e:
        print("[WARN] Could not enable TF32:", _e)

if getattr(cfg, 'use_bf16', False):
    bf16_ok = torch.cuda.is_bf16_supported()
    print(f"[DEVICE] bfloat16 support: {bf16_ok}")

print("Active config:")
for k, v in base_config.items():
    print(f"  {k}: {getattr(cfg, k)}")

In [ ]:
# Model, dataset and pipeline definitions (data + model init only)
import torch, os, json
from torch import nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import clip
from transformers import MT5ForConditionalGeneration, MT5Tokenizer

# (CUDA enforcement handled in config cell; assume cuda device later)

class ProjectionMLP(nn.Module):
    def __init__(self, in_dim, out_dim, prefix_tokens=8, hidden=1024, dropout=0.1):
        super().__init__()
        self.prefix_tokens = prefix_tokens
        self.fc1 = nn.Linear(in_dim, hidden)
        self.act = nn.GELU()
        self.fc2 = nn.Linear(hidden, out_dim * prefix_tokens)
        self.ln = nn.LayerNorm(out_dim)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        x = self.fc1(x); x = self.act(x); x = self.fc2(x)
        x = x.view(x.size(0), self.prefix_tokens, -1)
        x = self.ln(x)
        return self.dropout(x)

class CLIPmT5Pipeline(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        self.tokenizer = MT5Tokenizer.from_pretrained(cfg.model)
        self.model = MT5ForConditionalGeneration.from_pretrained(cfg.model)
        self.clip, _ = clip.load(cfg.clip_encoder, device='cpu')
        # CLIP freezing / unfreezing strategy
        if cfg.unfreeze_clip_last_n and cfg.unfreeze_clip_last_n > 0:
            # Freeze everything first
            for p in self.clip.parameters():
                p.requires_grad = False
            blocks = list(self.clip.visual.transformer.resblocks)
            for block in blocks[-cfg.unfreeze_clip_last_n:]:
                for p in block.parameters():
                    p.requires_grad = True
        else:
            # Respect freeze_clip flag (False means fully trainable)
            for p in self.clip.parameters():
                p.requires_grad = not cfg.freeze_clip
        # T5 freeze toggles
        if cfg.freeze_t5_encoder:
            for p in self.model.encoder.parameters():
                p.requires_grad = False
        if cfg.freeze_t5_decoder:
            for p in self.model.decoder.parameters():
                p.requires_grad = False
        self.prefix_tokens = cfg.prefix_tokens
        # Determine embedding dim dynamically (CLIP projection output)
        with torch.no_grad():
            dummy = torch.randn(1, 3, 224, 224)
            embed_dim = self.clip.visual.output_dim if hasattr(self.clip.visual, 'output_dim') else self.clip.encode_image(dummy).shape[-1]
        self.proj = ProjectionMLP(in_dim=embed_dim, out_dim=self.model.config.d_model, prefix_tokens=cfg.prefix_tokens)
        self._cached_sentinel_ids = None  # lazy cache

    def _encode_image_single(self, images: torch.Tensor):
        # (B, D) pooled embedding (already passed through ln_post + proj inside encode_image)
        pooled = self.clip.encode_image(images)
        return pooled

    def _encode_image_patch_tokens(self, images: torch.Tensor):
        """Return patch-averaged embedding projected into CLIP joint space.
        We manually replicate encode_image path but average patch tokens (excluding CLS),
        then apply ln_post and proj so final dim == visual.output_dim (e.g. 512) to match ProjectionMLP in_dim."""
        visual = self.clip.visual
        x = visual.conv1(images)                      # (B, width, grid, grid)
        x = x.reshape(x.shape[0], x.shape[1], -1)     # (B, width, patches)
        x = x.permute(0, 2, 1)                        # (B, patches, width)
        cls_tokens = visual.class_embedding.to(x.dtype) + torch.zeros(x.shape[0], 1, x.shape[-1], dtype=x.dtype, device=x.device)
        x = torch.cat([cls_tokens, x], dim=1)         # prepend CLS
        x = x + visual.positional_embedding.to(x.dtype)
        x = visual.ln_pre(x)
        x = x.permute(1, 0, 2)                        # (sequence, B, width)
        for block in visual.transformer.resblocks:
            x = block(x)
        x = x.permute(1, 0, 2)                        # (B, sequence, width)
        patches = x[:, 1:, :]                         # drop CLS
        pooled = patches.mean(dim=1)                  # (B, width)
        if hasattr(visual, 'ln_post'):
            pooled = visual.ln_post(pooled)
        if hasattr(visual, 'proj') and visual.proj is not None:
            pooled = pooled @ visual.proj             # (B, output_dim)
        return pooled

    def forward(self, images, src_texts, tgt_texts):
        device = next(self.parameters()).device
        images = images.to(device)
        clip_emb = self._encode_image_patch_tokens(images) if self.cfg.use_clip_patch_tokens else self._encode_image_single(images)
        prefix_emb = self.proj(clip_emb)
        tok_src = self.tokenizer(list(src_texts), return_tensors='pt', padding=True, truncation=True, max_length=self.cfg.src_max_len).to(device)
        tok_tgt = self.tokenizer(list(tgt_texts), return_tensors='pt', padding=True, truncation=True, max_length=self.cfg.tgt_max_len).to(device)
        text_emb = self.model.encoder.embed_tokens(tok_src.input_ids)
        full_emb = torch.cat([prefix_emb, text_emb], dim=1)
        full_attn = torch.cat([
            torch.ones(prefix_emb.size(0), self.prefix_tokens, dtype=tok_src.attention_mask.dtype, device=device),
            tok_src.attention_mask
        ], dim=1)
        return self.model(inputs_embeds=full_emb, attention_mask=full_attn, labels=tok_tgt.input_ids)

    def _prepare_prefix(self, images: torch.Tensor):
        images = images.to(next(self.parameters()).device)
        emb = self._encode_image_patch_tokens(images) if self.cfg.use_clip_patch_tokens else self._encode_image_single(images)
        return self.proj(emb)

    def _get_sentinel_bad_words(self, n=50):
        if self._cached_sentinel_ids is None:
            ids = [self.tokenizer(f'<extra_id_{i}>').input_ids[0] for i in range(n)]
            self._cached_sentinel_ids = [[i] for i in ids]
        return self._cached_sentinel_ids

    @torch.inference_mode()
    def generate(self, image_paths=None, images=None, num_beams=None, max_new_tokens=None, prompt="Bu görüntüyü açıkla: ", ban_sentinels=True, **gen_kwargs):
        device = next(self.parameters()).device
        num_beams = num_beams or self.cfg.num_beams_infer
        max_new_tokens = max_new_tokens or self.cfg.max_new_tokens_infer
        if images is None:
            assert image_paths is not None, "Provide image_paths or images tensor"
            preprocess = clip.load(self.cfg.clip_encoder, device='cpu')[1]
            pil_images = [Image.open(p).convert('RGB') for p in image_paths]
            images = torch.stack([preprocess(im) for im in pil_images])
        images = images.to(device)
        prefix_tokens = self._prepare_prefix(images)
        tok = self.tokenizer([prompt]*images.size(0), return_tensors='pt', padding=True, truncation=True, max_length=self.cfg.src_max_len).to(device)
        text_emb = self.model.encoder.embed_tokens(tok.input_ids)
        full_emb = torch.cat([prefix_tokens, text_emb], dim=1)
        full_attn = torch.cat([
            torch.ones(images.size(0), self.prefix_tokens, device=device, dtype=tok.attention_mask.dtype),
            tok.attention_mask
        ], dim=1)
        bad_words_ids = self._get_sentinel_bad_words() if ban_sentinels else None
        gen_ids = self.model.generate(
            inputs_embeds=full_emb,
            attention_mask=full_attn,
            num_beams=num_beams,
            max_new_tokens=max_new_tokens,
            bad_words_ids=bad_words_ids,
            **gen_kwargs
        )
        captions = [self.tokenizer.decode(g, skip_special_tokens=True).strip() for g in gen_ids]
        return [c if c else "<EMPTY>" for c in captions]

class Flickr8kCaptions(Dataset):
    def __init__(self, json_path, images_root, split=None, limit=None, clip_preprocess=None):
        self.images_root = images_root
        raw = json.load(open(json_path))
        rows = raw['images'] if isinstance(raw, dict) and 'images' in raw else raw
        self.samples = []
        for row in rows:
            if not isinstance(row, dict): continue
            if split and row.get('split') != split: continue
            img = row.get('filename') or row.get('image') or row.get('img')
            sentences = row.get('sentences')
            if not img or not sentences: continue
            for s in sentences:
                if isinstance(s, dict) and 'raw' in s:
                    self.samples.append((img, s['raw']))
        if limit: self.samples = self.samples[:limit]
        self.transform = clip_preprocess or clip.load(cfg.clip_encoder, device='cpu')[1]
    def __len__(self): return len(self.samples)
    def __getitem__(self, idx):
        img_name, cap = self.samples[idx]
        path = os.path.join(self.images_root, img_name)
        image = Image.open(path).convert('RGB')
        return self.transform(image), " ", cap

json_path = '/kaggle/input/tasviret/flickr8k/tasviret8k_captions.json'
images_root = '/kaggle/input/tasviret/flickr8k/Images'
train_dataset = Flickr8kCaptions(json_path, images_root, split='train', limit=cfg.dataset_limit)
val_dataset   = Flickr8kCaptions(json_path, images_root, split='val',   limit=None)
test_dataset  = Flickr8kCaptions(json_path, images_root, split='test',  limit=None)

train_loader = DataLoader(train_dataset, batch_size=cfg.batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=cfg.batch_size, shuffle=False) if len(val_dataset)>0 else None
test_loader  = DataLoader(test_dataset,  batch_size=cfg.batch_size, shuffle=False) if len(test_dataset)>0 else None

model_mm = CLIPmT5Pipeline(cfg)
print(f"Train samples: {len(train_dataset)}  Val: {len(val_dataset)}  Test: {len(test_dataset)}")
print("Trainable params:", sum(p.numel() for p in model_mm.parameters() if p.requires_grad))

In [ ]:
# Training cell (only training + validation)
import math, time, torch, warnings, os
from torch.optim import AdamW
from torch.optim.lr_scheduler import LambdaLR
warnings.filterwarnings("ignore", message=".*legacy behaviour.*MT5Tokenizer.*")

# CUDA-only enforcement (already asserted earlier, but double-check for isolation run)
assert torch.cuda.is_available(), "CUDA GPU required."
device = torch.device('cuda')
model_mm.to(device)

# Decide AMP dtype
amp_dtype = None
if getattr(cfg, 'use_amp', True):
    if getattr(cfg, 'use_bf16', False) and torch.cuda.is_bf16_supported():
        amp_dtype = torch.bfloat16
        print("[AMP] Using bfloat16 mixed precision")
    else:
        amp_dtype = torch.float16
        print("[AMP] Using float16 mixed precision")
else:
    print("[AMP] Disabled; using full float32")

# Parameter groups: separate CLIP vs non-CLIP for LR scaling
main_params = []
clip_params = []
for name, p in model_mm.named_parameters():
    if not p.requires_grad:
        continue
    if name.startswith('clip.'):
        clip_params.append(p)
    else:
        main_params.append(p)
param_groups = []
if main_params:
    param_groups.append({"params": main_params, "lr": cfg.lr})
if clip_params:
    scaled_lr = cfg.lr * getattr(cfg, 'clip_lr_scale', 0.05)
    param_groups.append({"params": clip_params, "lr": scaled_lr})
    print(f"[INFO] CLIP fine-tune params: {len(clip_params)} with lr={scaled_lr:.2e}")

optimizer = AdamW(param_groups, weight_decay=cfg.weight_decay)

steps_per_epoch = len(train_loader)
total_steps = steps_per_epoch * cfg.epochs

def lr_lambda(step):
    # linear warmup then cosine decay
    if cfg.warmup_steps > 0 and step < cfg.warmup_steps:
        return float(step) / float(max(1, cfg.warmup_steps))
    progress = (step - cfg.warmup_steps) / float(max(1, total_steps - cfg.warmup_steps))
    progress = min(max(progress, 0.0), 1.0)
    return 0.5 * (1 + math.cos(math.pi * progress))

scheduler = LambdaLR(optimizer, lr_lambda=lr_lambda)
scaler = torch.amp.GradScaler('cuda', enabled=amp_dtype is not None)
best_val = float('inf')
CKPT_DIR = 'checkpoints'; os.makedirs(CKPT_DIR, exist_ok=True)

# Early stopping state
early_patience = getattr(cfg, 'early_stop_patience', None)
min_delta = getattr(cfg, 'early_stop_min_delta', 0.0)
_epochs_no_improve = 0
stopped_early = False

monitor_history = []  # (epoch, metric)

global_step = 0
for epoch in range(cfg.epochs):
    model_mm.train()
    train_sum = 0.0
    t0 = time.time()
    for step, batch in enumerate(train_loader, start=1):
        imgs, srcs, tgts = batch
        imgs = imgs.to(device, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)

        if amp_dtype is not None:
            with torch.amp.autocast('cuda', dtype=amp_dtype):
                out = model_mm(imgs, srcs, tgts)
                loss = out.loss
            scaler.scale(loss).backward()
            if cfg.grad_clip and cfg.grad_clip > 0:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model_mm.parameters(), cfg.grad_clip)
            scaler.step(optimizer)
            scaler.update()
        else:
            out = model_mm(imgs, srcs, tgts); loss = out.loss
            loss.backward()
            if cfg.grad_clip and cfg.grad_clip > 0:
                torch.nn.utils.clip_grad_norm_(model_mm.parameters(), cfg.grad_clip)
            optimizer.step()

        scheduler.step()
        train_sum += loss.item()
        global_step += 1
    train_epoch_loss = train_sum / max(1, len(train_loader))

    val_epoch_loss = None
    if val_loader:
        model_mm.eval(); v = 0.0
        with torch.no_grad():
            for batch in val_loader:
                imgs, srcs, tgts = batch
                imgs = imgs.to(device, non_blocking=True)
                if amp_dtype is not None:
                    with torch.cuda.amp.autocast(dtype=amp_dtype):
                        out = model_mm(imgs, srcs, tgts)
                else:
                    out = model_mm(imgs, srcs, tgts)
                v += out.loss.item()
        val_epoch_loss = v / max(1, len(val_loader))

    dt = time.time() - t0
    current_lr = scheduler.get_last_lr()[0]
    if val_epoch_loss is not None:
        print(f"Epoch {epoch+1}/{cfg.epochs} train_loss={train_epoch_loss:.4f} val_loss={val_epoch_loss:.4f} time={dt:.1f}s lr={current_lr:.2e}", flush=True)
        if use_wandb:
            wandb.log({"train/epoch_loss": train_epoch_loss, "val/epoch_loss": val_epoch_loss, "lr": current_lr}, step=epoch)
    else:
        print(f"Epoch {epoch+1}/{cfg.epochs} train_loss={train_epoch_loss:.4f} time={dt:.1f}s lr={current_lr:.2e}", flush=True)
        if use_wandb:
            wandb.log({"train/epoch_loss": train_epoch_loss, "lr": current_lr}, step=epoch)

    metric = val_epoch_loss if val_epoch_loss is not None else train_epoch_loss
    monitor_history.append((epoch, metric))

    improved = metric < (best_val - min_delta)
    if improved:
        best_val = metric
        _epochs_no_improve = 0
        torch.save({
            'model': model_mm.state_dict(),
            'cfg': {k: getattr(cfg, k) for k in base_config.keys()},
            'epoch': epoch,
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.state_dict(),
            'best_val': best_val,
        }, os.path.join(CKPT_DIR, 'best.pt'))
        print(f"  -> Saved checkpoint (metric {best_val:.4f})")
    else:
        _epochs_no_improve += 1

    if early_patience is not None and _epochs_no_improve >= early_patience:
        print(f"[Early Stop] No improvement (>{min_delta} delta) for {early_patience} consecutive epochs. Stopping at epoch {epoch+1}.")
        stopped_early = True
        break

print("Training finished. Best metric:", best_val, "(early stop)" if stopped_early else "")

In [ ]:
# Testing / Inference cell (evaluate test set + simple generation API)
import torch, time, os, json, math
from typing import List
from PIL import Image

assert torch.cuda.is_available(), "CUDA GPU required for inference."
model_mm.eval()
device = torch.device('cuda')
model_mm.to(device)

# Determine AMP dtype consistent with training
amp_dtype = None
if getattr(cfg, 'use_amp', True):
    if getattr(cfg, 'use_bf16', False) and torch.cuda.is_bf16_supported():
        amp_dtype = torch.bfloat16
    else:
        amp_dtype = torch.float16

test_loss = None
if test_loader:
    t0 = time.time(); total=0.0
    with torch.no_grad():
        for batch in test_loader:
            imgs, srcs, tgts = batch
            imgs = imgs.to(device, non_blocking=True)
            if amp_dtype is not None:
                with torch.cuda.amp.autocast(dtype=amp_dtype):
                    out = model_mm(imgs, srcs, tgts)
            else:
                out = model_mm(imgs, srcs, tgts)
            total += out.loss.item()
    test_loss = total / max(1,len(test_loader))
    print(f"Test loss={test_loss:.4f} time={(time.time()-t0):.1f}s")
else:
    print("No test split available.")

############################################
# Caption Quality Metrics (BLEU1-4, METEOR, CIDEr, ROUGE-L) + W&B logging
############################################
metrics = {}
try:
    # Build reference sets (all captions per test image) from original JSON
    # Assumes json_path & test_dataset defined in earlier cell
    with open(json_path) as f:
        data_json = json.load(f)
    img_entries = data_json['images'] if isinstance(data_json, dict) else data_json

    # Gather unique test image filenames present in test_dataset
    test_image_names = sorted({s[0] for s in test_dataset.samples})
    refs_map = {}
    for entry in img_entries:
        if not isinstance(entry, dict): 
            continue
        fname = entry.get('filename')
        if fname in test_image_names:
            # collect all raw captions
            all_caps = []
            for s in entry.get('sentences', []):
                if isinstance(s, dict) and 'raw' in s:
                    cap = s['raw'].strip()
                    if cap:
                        all_caps.append(cap)
            if all_caps:
                refs_map[fname] = all_caps

    # Generate hypotheses (batched for speed)
    hyps = []
    refs = []
    BATCH_GEN = 8
    image_root = images_root  # already defined earlier
    image_paths = [os.path.join(image_root, fname) for fname in test_image_names]

    def _generate_batch(paths):
        if amp_dtype is not None:
            with torch.cuda.amp.autocast(dtype=amp_dtype):
                return model_mm.generate(image_paths=paths, ban_sentinels=True)
        return model_mm.generate(image_paths=paths, ban_sentinels=True)

    for i in range(0, len(image_paths), BATCH_GEN):
        batch_paths = image_paths[i:i+BATCH_GEN]
        caps = _generate_batch(batch_paths)
        for p, pred in zip(batch_paths, caps):
            fname = os.path.basename(p)
            hyps.append(pred)
            refs.append(refs_map.get(fname, [""]))  # list of refs

    # Install/load evaluate (quiet) if missing
    try:
        import evaluate
    except ImportError:
        import subprocess, sys
        subprocess.run([sys.executable, "-m", "pip", "install", "-q", "evaluate"], check=False)
        import evaluate

    # Load metrics
    bleu_metric = evaluate.load("bleu")
    meteor_metric = evaluate.load("meteor")
    rouge_metric = evaluate.load("rouge")
    cider_metric = None
    try:
        cider_metric = evaluate.load("cider")
    except Exception:
        print("[WARN] CIDEr metric load failed (pycocoevalcap might not be available).")

    # Compute
    bleu_res = bleu_metric.compute(predictions=hyps, references=refs)
    meteor_res = meteor_metric.compute(predictions=hyps, references=refs)
    rouge_res = rouge_metric.compute(predictions=hyps, references=[" ".join(r[0].split()) if isinstance(r,list) and len(r)==1 else r[0] for r in refs])  # rouge expects single ref; use first
    cider_score = None
    if cider_metric:
        # cider expects list[str] predictions and list[list[str]] references
        cider_res = cider_metric.compute(predictions=hyps, references=refs)
        cider_score = cider_res.get("score")

    metrics = {
        "bleu1": bleu_res["precisions"][0],
        "bleu2": bleu_res["precisions"][1],
        "bleu3": bleu_res["precisions"][2],
        "bleu4": bleu_res["precisions"][3],
        "bleu": bleu_res["bleu"],
        "meteor": meteor_res["meteor"],
        "rougeL": rouge_res.get("rougeL", float('nan')),
    }
    if cider_score is not None:
        metrics["cider"] = cider_score
    if test_loss is not None:
        metrics["test_loss"] = test_loss

    print("=== Caption Metrics ===")
    for k,v in metrics.items():
        print(f"{k}: {v:.4f}" if isinstance(v,(int,float)) else f"{k}: {v}")

    if use_wandb:
        # Namespace under eval/
        wandb.log({f"eval/{k}": v for k,v in metrics.items()}, commit=True)
        print("[wandb] Logged caption metrics.")

except Exception as e:
    print("[ERROR] Metric computation failed:", e)

# Show a few sample generated captions from first N test images with diversity controls
default_gen_kwargs = dict(
    repetition_penalty=1.2,
    no_repeat_ngram_size=3,
)
SAMPLE_PRINTS = 5
if test_loader and SAMPLE_PRINTS > 0:
    shown = 0
    printed_imgs = set()
    for img_path, _ in [(os.path.join('/kaggle/input/tasviret/flickr8k/Images', s[0]), s[1]) for s in test_dataset.samples]:
        if img_path in printed_imgs:
            continue
        if amp_dtype is not None:
            with torch.cuda.amp.autocast(dtype=amp_dtype):
                caps = model_mm.generate(image_paths=[img_path], **default_gen_kwargs)
        else:
            caps = model_mm.generate(image_paths=[img_path], **default_gen_kwargs)
        gt_caption = next(s[1] for s in test_dataset.samples if os.path.join(images_root, s[0]) == img_path)
        print('GT:', gt_caption)
        print('CAP:', caps[0])
        print('---------------')
        printed_imgs.add(img_path)
        shown += 1
        if shown >= SAMPLE_PRINTS:
            break

# Optional: stochastic sampling helper for more diverse outputs
def generate_captions(image_paths: List[str], mode='beam', **kwargs):
    if mode == 'sample':
        sample_defaults = dict(temperature=0.8, top_p=0.9, do_sample=True, repetition_penalty=1.15, no_repeat_ngram_size=3)
        for k, v in sample_defaults.items():
            kwargs.setdefault(k, v)
    else:  # beam
        beam_defaults = dict(repetition_penalty=1.2, no_repeat_ngram_size=3)
        for k, v in beam_defaults.items():
            kwargs.setdefault(k, v)
    if amp_dtype is not None:
        with torch.cuda.amp.autocast(dtype=amp_dtype):
            return model_mm.generate(image_paths=image_paths, **kwargs)
    return model_mm.generate(image_paths=image_paths, **kwargs)

print("[Ready] generate_captions(['path/to/img.jpg'], mode='beam'")

In [ ]:
# Single Image Inference
# --- Predict helper (single image) ---
from typing import Optional, Dict, Any, List
import matplotlib.pyplot as plt

def predict(
    image_path: str,
    prompt: str = "Bu fotoğrafı açıkla: ",
    mode: str = "beam",              # 'beam' or 'sample'
    max_refs: Optional[int] = 5,
    show_image: bool = True,
    show_refs: bool = True,
    gen_kwargs: Optional[Dict[str, Any]] = None,
    json_file: Optional[str] = None,
    ban_sentinels: bool = True
) -> Dict[str, Any]:
    """
    Generate a caption for one image and (optionally) display it with reference captions.

    Returns dict with: image_path, prediction, references (list), mode.
    """
    assert os.path.isfile(image_path), f"Image not found: {image_path}"
    jf = json_file or json_path  # use global json_path defined earlier

    # Collect reference captions (all captions for this filename)
    refs: List[str] = []
    try:
        with open(jf) as f:
            data = json.load(f)
        entries = data['images'] if isinstance(data, dict) and 'images' in data else data
        target_name = os.path.basename(image_path)
        for e in entries:
            if not isinstance(e, dict):
                continue
            if e.get('filename') == target_name:
                for s in e.get('sentences', []):
                    if isinstance(s, dict) and 'raw' in s:
                        cap = s['raw'].strip()
                        if cap:
                            refs.append(cap)
                break
    except Exception as e:
        print(f"[WARN] Could not parse references ({e})")

    if max_refs is not None:
        refs = refs[:max_refs]

    # Defaults for decoding
    gen_kwargs = (gen_kwargs or {}).copy()
    if mode == "sample":
        defaults = dict(temperature=0.8, top_p=0.9, do_sample=True,
                        repetition_penalty=1.15, no_repeat_ngram_size=3)
    else:
        defaults = dict(repetition_penalty=1.2, no_repeat_ngram_size=3,
                        num_beams=getattr(cfg, "num_beams_infer", 4))
    for k, v in defaults.items():
        gen_kwargs.setdefault(k, v)

    # Use global mixed precision dtype if available
    local_amp_dtype = globals().get("amp_dtype", None)
    model_mm.eval()
    if local_amp_dtype is not None:
        with torch.cuda.amp.autocast(dtype=local_amp_dtype):
            pred = model_mm.generate(
                image_paths=[image_path],
                prompt=prompt,
                ban_sentinels=ban_sentinels,
                **gen_kwargs
            )[0]
    else:
        pred = model_mm.generate(
            image_paths=[image_path],
            prompt=prompt,
            ban_sentinels=ban_sentinels,
            **gen_kwargs
        )[0]

    # Visualization
    if show_image:
        try:
            import matplotlib.pyplot as plt
            img = Image.open(image_path).convert("RGB")
            plt.figure(figsize=(5,5))
            plt.imshow(img); plt.axis("off")
            plt.title("Image")
            plt.show()
        except Exception as e:
            print(f"[WARN] Could not display image ({e})")

    if show_refs:
        if refs:
            print("References:")
            for i, r in enumerate(refs, 1):
                print(f"  {i}. {r}")
        else:
            print("(No references found)")
    print("Prediction:")
    print(" ", pred)

In [ ]:
result = predict("/kaggle/input/tasviret/flickr8k/Images/1032460886_4a598ed535.jpg", mode='beam')
print(result)